## (Упрощенная) ERD сущностей в Clickup
![test](ClickupERD.png "Title")

 Команда - основная верхнеуровневая сущность. Название говорит само за себя. Нас интересует исключительно в разрезе team_id, который используется в некоторых эндпойнтах API
 В команде может быть несколько воркспейсов, в которых, в свою очередь, содержатся листы, которые могут быть сгруппированы в папки, но могут быть и прямыми детьми воркспейса.
 Лист - основная сущность, интересующая нас в контексте сборки датасетов, т.к. это коллекция тасков, которые, в свою очередь, содержат всю интересующую нас информацию (дата создания, статус и т.п.). Стоит обратить внимание, что мы можем определить кастомные поля для использования в тасках (например, причина отвала и т.п.). При запросе к API эти поля будут доступны по отдельному ключу custom_fields.
В интерфейсе все это выглядит как-то так:

![Interface](Entities.png "Ent")

In [1]:
import pandas as pd
import json
import ClickUpAPI as cua
import numpy as np
from ast import literal_eval
from math import ceil
from typing import List
from urllib.parse import quote_plus

Первым делом, как и для практически любого API, нам понадобится токен. В Кликапе есть два вида токенов: персональный, который можно получить в секции Apps настроек своего профиля и полноценный authorization code grant OAuth2. Здесь мы будем использовать персональный токен. ![SegmentLocal](PersToken3.gif "segment")

In [2]:
token = os.environ.get("ClickUpToken")

In [3]:
# Проверим, правильно ли API возвращает юзера, от лица которого делаются запросы:
test_class = cua.ClickupClient(token)
res = test_class.get_user()
print(res)

{'id': 8867829, 'username': 'Bogdan Pilyavets', 'email': 'rupilbo@yandex-team.ru', 'color': '#795548', 'profilePicture': None, 'initials': 'BP', 'week_start_day': None, 'global_font_support': False, 'timezone': 'Europe/Moscow'}


In [4]:
# Получим айдишник команды, к которой у нас есть доступ:
team_id = test_class.get_teams(id_only=True)
print(team_id)

4514698


In [5]:
# получим всю инфу про интересующий нас спейс (в данном случае CRM)
space = test_class.get_space_by_name(team_id, name='CRM')
print(space)

{'id': '6849848', 'name': 'CRM', 'private': False, 'statuses': [{'id': 'p6849848_hGuYhhYV', 'status': 'Open', 'type': 'open', 'orderindex': 0, 'color': '#d3d3d3'}, {'id': 'p6849848_aJ5x29At', 'status': 'concept', 'type': 'custom', 'orderindex': 1, 'color': '#FFCC00'}, {'id': 'p6849848_uHca5I69', 'status': 'in progress', 'type': 'custom', 'orderindex': 2, 'color': '#FF540D'}, {'id': 'p6849848_rQtXfgXn', 'status': 'running', 'type': 'custom', 'orderindex': 3, 'color': '#FF00DF'}, {'id': 'p6849848_Yuo3uSOo', 'status': 'review', 'type': 'custom', 'orderindex': 4, 'color': '#A875FF'}, {'id': 'p6849848_e0PVKVnw', 'status': 'Closed', 'type': 'closed', 'orderindex': 5, 'color': '#6bc950'}], 'multiple_assignees': True, 'features': {'due_dates': {'enabled': True, 'start_date': True, 'remap_due_dates': False, 'remap_closed_due_date': False}, 'sprints': {'enabled': True}, 'time_tracking': {'enabled': True, 'harvest': False, 'rollup': False}, 'points': {'enabled': True}, 'custom_items': {'enabled':

In [6]:
# получим всю инфу про интересующие нас листы (в данном случае 'Учащиеся' и 'Лиды (родители)')
studs = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Учащиеся')
pars = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Лиды (родители)')

In [7]:
tags = test_class.get_tags(space['id'])
tags_names = [tag['name'] for tag in tags['tags']]
tags_to_keep = [quote_plus(tag) for tag in tags_names if tag not in ['тестовый', 'технический']]
tags_to_keep.append('')

In [8]:
#заберем данные по таскам 
tasks = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[pars['id']])

page=0&list_ids[]=44610695&include_closed=true
page=1&list_ids[]=44610695&include_closed=true
page=2&list_ids[]=44610695&include_closed=true
page=3&list_ids[]=44610695&include_closed=true
page=4&list_ids[]=44610695&include_closed=true
page=5&list_ids[]=44610695&include_closed=true
page=6&list_ids[]=44610695&include_closed=true
page=7&list_ids[]=44610695&include_closed=true
page=8&list_ids[]=44610695&include_closed=true
page=9&list_ids[]=44610695&include_closed=true
page=10&list_ids[]=44610695&include_closed=true
page=11&list_ids[]=44610695&include_closed=true
page=12&list_ids[]=44610695&include_closed=true
page=13&list_ids[]=44610695&include_closed=true
page=14&list_ids[]=44610695&include_closed=true
page=15&list_ids[]=44610695&include_closed=true
page=16&list_ids[]=44610695&include_closed=true
page=17&list_ids[]=44610695&include_closed=true


In [9]:
#поскольку get_all_tasks возвращает нам список nested джейсонов, напишем функцию, которая сложит все это добро в один датафрейм
#подробнее тут https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
def normalize_wrapper(data:List[dict], **kwargs) -> pd.DataFrame:
    """
    replaces the empty list with a dummy dictionary
    """
    for i in data:
        if not i['tags']:
            i['tags']=[{'name': "fake"}]  #TODO параметризовать ключи
    return pd.json_normalize(data, **kwargs)

def df_getter(data:List[dict], key_getter='tasks', **kwargs) -> pd.DataFrame:
    """
    transforms a list of jsons into a single dataframe using pd.json_normalize
    """
    res = list()
    for chunk in data:
        res.append(normalize_wrapper(chunk[key_getter], **kwargs))
    return pd.concat(res)

In [10]:
par_names = df_getter(tasks,
                      record_path='custom_fields',
                      meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                           ['status', 'status'], ['status', 'type'],
                           ['creator', 'email'], 'due_date', 'start_date', 'tags'],
                      meta_prefix='standard_')
#в последней странице почему-то остается только словарь, сейчас нет времени разбираться
missing_list = par_names.loc[par_names['standard_tags']=={'name': 'fake'}]     
par_names.loc[par_names['standard_tags']=={'name': 'fake'}, 'standard_tags'] = pd.Series([[{'name': 'fake'}]]*len(missing_list))

In [11]:
# вытащим теги, чтобы потом выкинуть тестовиков
tags = par_names[['standard_tags', 'standard_id']]
#отберем релевантные столбцы и пивотнем кастомные поля
par_names = par_names[par_names['name'].isin(['Источник трафика', 'Канал привлечения', 'причина отказа', 'Дата заявки'])][['name', 'value', 'standard_name', 'standard_id', 'standard_text_content']]
par_names = par_names.pivot(index=['standard_name', 'standard_id', 'standard_text_content'], columns='name', values='value').reset_index()
par_names = par_names.set_index('standard_id')
#резберем строку с utm-метками и превратим ее в 3 столбца
par_names['Источник трафика'] = par_names['Источник трафика'].fillna("")
par_names['utms'] = par_names['Источник трафика'].str.findall(r"(?<==)([\w\-_\s]*)")
par_names['keys'] = par_names['Источник трафика'].str.findall(r"([\w\-_]+)(?==)")
#astype здесь используется как способ конвертнуть пустые и непустые листы в False и True соответственно
mask = par_names['keys'].astype(bool)
antimask = par_names['keys'].astype(bool) == False
utms = par_names.loc[mask, ('utms', 'keys')]
no_utms = par_names.loc[antimask, ('Источник трафика', 'utms')]
#Конструируем словарь utm-ок для тех у кого они заполнены
utms['dict'] = utms.apply(lambda x: {x['keys'][i]:x['utms'][i] for i in range(len(x['keys']))}, axis=1)
#Конструируем словарь utm-ок по умолчанию, для тех у кого они не прокинуты
no_utms['dict'] = no_utms.apply(lambda x: {"utm_source":x['Источник трафика']}, axis=1)
#Собираем это воедино, разбиваем на столбцы и джойним к родительскому датафрейму
full_utms = pd.concat([utms['dict'], no_utms['dict']])
stand_index = full_utms.index
full_utms = pd.DataFrame(full_utms.tolist())
full_utms = full_utms.set_index(stand_index)
par_names = pd.concat([par_names, full_utms], axis=1)

In [12]:
#отдельно обработаем случай с автоматически сгенеренными лидами с fb
par_names.loc[par_names['Источник трафика']=='fb_leads', 'utm_campaign'] = 'fb_leads'
#par_names.loc[par_names['standard_text_content'].str.contains('fb_leads', na=False), ('utm_source', 'utm_campaign')] = ('fb', 'leads')
par_names = par_names.reset_index().rename({'index': 'standard_id'}, axis=1)
par_names = par_names[['standard_name', 'standard_id', 'utm_source', 'utm_campaign', 'utm_term', 'utm_medium', 'utm_content', 'причина отказа', 'Дата заявки']]
par_names['standard_name'] = par_names['standard_name'].replace(r'\n', ' ', regex=True)
#Конвертнем дату заявки в дату
par_names['Дата заявки'] = (par_names['Дата заявки'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")).dt.date

In [13]:
#сформируем список тестовых айдишников
tags['standard_tags'] = tags['standard_tags'].astype(str)
throwaway=tags.loc[tags['standard_tags'].str.contains('тестовый|технический'), 'standard_id'].unique()

In [14]:
#получим плоский лист всех родительских айдишников
parent_ids = np.setdiff1d(par_names['standard_id'].unique(), throwaway)
task_ids = list(parent_ids)
history = test_class.get_time_in_status(task_ids)

In [15]:
#поскольку get_time_in_status возвращает нам список словарей,
#в роли ключей у которых выступают родительские айдишники
#соберем это в единый словарь и превратим в датафрейм
flattened = dict()
for task_collection in history:
    flattened.update(task_collection)
flat_history = {task_id:flattened[task_id]['status_history'] for task_id in flattened}
flat_df = pd.DataFrame.from_dict(flat_history, orient='index')
#сейчас у нас столбцы со словарями для истории каждого статуса - пометим их префиксом
columns = ['col'+str(i) for i in range(len(flat_df.columns))]
flat_df.columns = columns
flat_df = flat_df.reset_index()
#разберем вложенные словари в каждой группе столбцов через json_normalize
#и дропнем столбцы со словарями
flat_df = flat_df.where(flat_df.notna(), lambda x: [{}])
for col in columns:
    flat_df = flat_df.join(pd.json_normalize(flat_df[col]).add_prefix(col+'_'))
flat_df = flat_df.drop(columns=columns)
flat_df.head()


,index,col0_status,col0_color,col0_type,col0_orderindex,col0_total_time.by_minute,col0_total_time.since,col1_status,col1_color,col1_type,...,col13_status,col13_color,col13_type,col13_total_time.by_minute,col13_total_time.since,col14_status,col14_color,col14_type,col14_total_time.by_minute,col14_total_time.since
0,119e7p5,0. новый лид,#d3d3d3,open,0,604,1625603197254,3.слот встречи назначен,#f9d900,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,119eteq,0. новый лид,#d3d3d3,open,0,566,1625606086585,2.перезвонить на неделе,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119fjhx,0. новый лид,#d3d3d3,open,0,510,1625609723280,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119g3bp,0. новый лид,#d3d3d3,open,0,425,1625615578207,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,119gcy3,0. новый лид,#d3d3d3,open,0,366,1626273866613,1. недозвон (нет контакта),#800000,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#превратим нашу таблицу из широкой в длинную и избавимся от префиксов для столбцов с одинановой сутью
#например, для time_by_minute (время в статусе)
melted_df = flat_df.fillna("").melt(id_vars=['index'])
melted_df['variable'] = melted_df['variable'].str.split("_", n=1).str[1]
#выкинем бесполезные столбцы и сделаем родительские айдишники индексами
melted_df = melted_df[~melted_df['variable'].isin(['orderindex', 'color'])]
melted_df = melted_df.set_index('index')
#соберем список отдельных датафреймов для каждого статуса и схлопнем их в одну таблицу
#NB при итерировании по групбаю он возвращает сначала ключ, а потом датафрейм по этому ключу
to_concat = [y for x,y in melted_df.groupby('variable', as_index=False)]
new_df = pd.concat(to_concat, axis=1)
#дропнем нерелевантные столбцы и переименуем оставшиеся
col_names = list(new_df.iloc[0, [i for i in range(0,len(new_df.columns),2)]])
new_df = new_df.iloc[:,[i for i in range(1,len(new_df.columns),2)]]
new_df.columns = col_names
new_df = new_df.rename(columns={'total_time.by_minute':'min_passed', 'total_time.since': 'start_dt'})
#конвертнем столбец со временем в формат даты
new_df['start_dt'] = new_df['start_dt'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")
new_df.head()

,status,min_passed,start_dt,type
index,,,,
119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open
119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open
119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open
119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open
119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open


In [17]:
#получим список всех детей из кликапа
kids = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[studs['id']])

page=0&list_ids[]=46598432&include_closed=true
page=1&list_ids[]=46598432&include_closed=true
page=2&list_ids[]=46598432&include_closed=true


In [18]:
#как и родителями, сложим все в один датафрейм
df = df_getter(kids,
               record_path='custom_fields',
               meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                    ['status', 'status'], ['status', 'type'],
                    ['creator', 'email'], 'due_date', 'start_date'],
               meta_prefix="standard_")
df.columns

Index(['id', 'name', 'type', 'date_created', 'hide_from_guests', 'required',
       'value', 'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id',
       'type_config.options', 'standard_id', 'standard_name',
       'standard_text_content', 'standard_date_created',
       'standard_date_updated', 'standard_date_closed',
       'standard_status.status', 'standard_status.type',
       'standard_creator.email', 'standard_due_date', 'standard_start_date'],
      dtype='object')

In [19]:
#выкинем тестовых юзеров
df = df[df['standard_status.status']!='тестовый']
df.sample(5)

,id,name,type,date_created,hide_from_guests,required,value,type_config.fields,type_config.field_inverted_name,type_config.linked_subcategory_access,...,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
678,b82a5972-5fb8-4be4-bcef-af0fbc272c05,Shared with me,list_relationship,1620249016705,False,False,NaN,[],NaN,False,...,София Соловьева,\n,1626872298254,1626954058895,None,возврат,custom,rahmatulina@yandex-team.ru,None,None
911,97b1b7e0-eccd-4a92-abfc-829147a4cbc5,первое занятие,date,1613757223046,False,False,1613523600000,NaN,NaN,NaN,...,Вася Кузнецов,None,1613383673135,1616411576244,1616411576244,отвал (скрыть ученика),closed,irawhitelake@gmail.com,None,None
15,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'id': 'j33grw', 'name': 'Анна Диганова', 'st...",[{'field': 'cf_b261ecb0-58b9-4a4e-bcae-1e1cafb...,Учащийся,True,...,Олег Диганов,\n,1628772207379,1628772239026,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
391,bc6f57cf-5ba5-4649-aeee-0c2f0b0250ed,Прогулы,list_relationship,1627581353901,False,False,NaN,[],NaN,True,...,Илья Курило,\n,1622462256200,1625833331642,None,не планирует покупку,custom,rahmatulina@yandex-team.ru,None,None
877,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,NaN,NaN,NaN,...,Саша Яковлев,\n,1626425489932,1626425756685,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None


In [20]:
#удалим не нужные нам столбцы
to_drop = ['type_config.default', 'type_config.placeholder', 'type_config.new_drop_down', 'type_config.options',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id']
df = df.drop(columns=df.columns.intersection(to_drop))
df.head()

,id,name,type,date_created,hide_from_guests,required,value,standard_id,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
0,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,NaN,u3vf2h,Виктория Семенюк,\n,1628792862692,1628792881421,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
1,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,u3vf2h,Виктория Семенюк,\n,1628792862692,1628792881421,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
2,4a4363cf-e27e-4b59-b781-63cb161a43e3,Расписание,short_text,1613142022298,False,False,NaN,u3vf2h,Виктория Семенюк,\n,1628792862692,1628792881421,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
3,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'id': 'kfb0ry', 'name': 'Лариса Семенюк', 's...",u3vf2h,Виктория Семенюк,\n,1628792862692,1628792881421,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
4,e6cb4ac4-e138-4895-968d-9ebccaa12c78,Согласие на видеозапись,labels,1613132651997,False,False,[8472ccaa-0c6e-4537-b71c-f2345ec26b11],u3vf2h,Виктория Семенюк,\n,1628792862692,1628792881421,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None


In [21]:
#пивотнем столбец name (каждое уникальное значение из этого столбца становится отдельным столбцом в пивотизированном датафрейме)
#поскольку столбец 'родитель в воронке лидов' в кликапе является ссылкой на другой лист, API возвращает нам этот лист, а пандас
#читает его как питоновский лист. В нашем случае в нем всегда один элемент, поэтому explode не увеличивает кол-во строк в датафрейме
#а просто избавляет нас от листа в cтолбце, превращая ее в столбец словарей
df2 = df.copy()
df2 = df2.pivot(index='standard_id', columns='name', values='value')
df2 = df2.explode('Родитель в воронке лидов')
df2.head()

name,Shared with me,Инциденты,Преподаватель списка учащиеся,Прогулы,Пройденные занятия пакета (старое),Расписание,Родитель в воронке лидов,Согласие на видеозапись,Цель ученика и сроки достижения цели,класс,купленные пакеты,первое занятие
standard_id,,,,,,,,,,,,
6fynn4,[],NaN,Дарья Карпухина,NaN,11\02 перерыв с 22\02 до 28\02,"чт 18-00 - 60 мин, пн 20-00 - 15 мин","{'id': '69yhrv', 'name': 'Татьяна Рушечникова ...",[1171db5a-754c-4178-85f1-67847ab15f03],"Дочь 6 класс, не очень любит математику, после...",NaN,"[{'id': 'ejzw6t', 'name': 'Варя Рушечникова па...",1613005200000
6fynn5,NaN,NaN,"Светлана Зайцева (15 мин), Сергей Христолюбов ...",NaN,16\02,"вт 16-00 60 мин, чт 16-00 15 мин","{'id': '6fu8tt', 'name': 'Анна Гуляева', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6w', 'name': 'Иван Гуляев пакет 4...",1613437200000
6fyppm,NaN,NaN,"Светлана Зайцева (15 мин), Анна И (60 мин)",NaN,NaN,"вскр 14-00 - 60 мин, вт 16-00 - 15 мин","{'id': 'd6zdz6', 'name': 'Ольга', 'status': '9...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'fk0rm6', 'name': 'Даша пакет 4+4 купл...",1612054800000
6hqz5e,[],NaN,Антон Загривин,"[{'id': 'rp4ytc', 'name': 'Аня Таран', 'status...",15\02,"пн 12-00 60 мин, чт 12-00 15 мин","{'id': '6dy7aj', 'name': 'Марина Таран', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6r', 'name': 'Аня пакет 4+4 купле...",1613350800000
6huakn,NaN,NaN,Света и Никита,NaN,27\02,"среда 17-00 60 мин, пятница 18-30 15 мин","{'id': '6fut3z', 'name': 'Наталья Кузнецова', ...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6c', 'name': 'Вася Кузнецов пакет...",1613523600000


In [22]:
#читаем наш столбец со словарями в отдельный датафрейм, добавляем префикс к именам столбцов
#чтобы избежать конфликтов при последующем мердже
parent = df2['Родитель в воронке лидов'].apply(pd.Series).add_prefix("par_")
parent = parent[['par_id', 'par_name']]
parent.head()

,par_id,par_name
standard_id,,
6fynn4,69yhrv,Татьяна Рушечникова (Болгова)
6fynn5,6fu8tt,Анна Гуляева
6fyppm,d6zdz6,Ольга
6hqz5e,6dy7aj,Марина Таран
6huakn,6fut3z,Наталья Кузнецова


In [23]:
#проделываем то же самое для столбца со ссылкой на купленные пакеты
packages = df2['купленные пакеты'].explode().apply(pd.Series).add_prefix("purch_")
packages = packages[['purch_id']]
packages.head()

,purch_id
standard_id,
6fynn4,ejzw6t
6fynn4,71t1d4
6fynn4,7dyck9
6fynn4,gz8w39
6fynn4,gk7nub


In [24]:
#получаем инфу о списке покупок в кликапе
purch_list = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Заявки на покупку')
print(purch_list)

{'id': '48853634', 'name': 'Заявки на покупку', 'orderindex': 0, 'status': None, 'priority': None, 'assignee': None, 'task_count': 287, 'due_date': None, 'start_date': None, 'folder': {'id': '23627065', 'name': 'hidden', 'hidden': True, 'access': True}, 'space': {'id': '6849848', 'name': 'CRM', 'access': True}, 'archived': False, 'override_statuses': True, 'permission_level': 'create'}


In [25]:
#забираем все таски из этого списка
purchases = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids = [purch_list['id']])

page=0&list_ids[]=48853634&include_closed=true
page=1&list_ids[]=48853634&include_closed=true
page=2&list_ids[]=48853634&include_closed=true
page=3&list_ids[]=48853634&include_closed=true
page=4&list_ids[]=48853634&include_closed=true


In [26]:
#и внвь соберем эту инфу в плоский датафрейм, выкинем тесовых и оставим только релевантные столбцы
purch_df = df_getter(purchases, record_path='custom_fields', meta=['id', ['status', 'status']],meta_prefix='standard_')
purch_df = purch_df[(purch_df['name']=='Ссылка на ребенка в админке') & (purch_df['standard_status.status']!='тестовый')][['value', 'standard_id']].dropna()
#вытащим praktikum_id из юрла
purch_df['praktikum_id'] = purch_df['value'].str.split("/").str[-1]
purch_df.head()

,value,standard_id,praktikum_id
3,https://praktikum-admin.yandex-team.ru/math/st...,u3wdkp,12062574
13,https://praktikum-admin.yandex-team.ru/math/st...,u3veum,12055643
23,https://praktikum-admin.yandex-team.ru/math/st...,u3rtvj,12046862
33,https://praktikum-admin.yandex-team.ru/math/st...,u3qqz9,10937593
43,https://praktikum-admin.yandex-team.ru/math/st...,u3pr2w,12042389


In [27]:
#сопоставим айдишники детей с их айдишниками в практикуме
kids_links = packages.reset_index().merge(purch_df, how='left', left_on='purch_id', right_on='standard_id')
kids_links = kids_links[['purch_id', 'praktikum_id', 'standard_id_x']].groupby('standard_id_x')['praktikum_id'].last()
kids_links.head()

standard_id_x
6fynn4    7210563
6fynn5    7203187
6fyppm       None
6hqz5e    7245837
6huakn       None
Name: praktikum_id, dtype: object

In [28]:
#теперь сджойним детей с их родителями
full_lineage = parent.merge(kids_links, how='left', left_index=True, right_index=True)
full_lineage.sample(5)

,par_id,par_name,praktikum_id
standard_id,,,
pzgg4y,pqfygk,Ирина Перстнева,11731447
p3c1ur,119g3bp,Анна(Денис),11200929
n18dtf,n188x5,Полина Перемитина,10276897
h348ut,gz76ew,Даниил Киселёв,7431678
nzfvtw,11bjdrg,Елена (Иосиф),11157305


In [29]:
#наконец, смерджим все это воедино с датафреймом родительских атрибутов
fin_df = new_df.reset_index().merge(full_lineage, how='left', left_on='index', right_on='par_id')
fin_df = fin_df[fin_df['type']!=""]
fin_df = fin_df.merge(par_names, how='left', left_on='index', right_on='standard_id')
fin_df.sample(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_term,utm_medium,utm_content,причина отказа,Дата заявки
6868,hx7ghz,новый лид,672.0,2021-04-27 23:45:10.701,open,hx7ghz,Евгения Винер,7772528,Евгения Винер,hx7ghz,other2,NaN,NaN,NaN,NaN,NaN,2021-04-27
631,6dt3m7,2. отказ после контакта,76153,2021-06-21 13:14:29.225,custom,NaN,NaN,NaN,Ольга Яковлева,6dt3m7,facebook,NaN,NaN,NaN,NaN,15,2021-02-02
4054,jzc3xb,5.встреча подтверждена,300.0,2021-05-19 15:12:53.326,custom,jzc3xb,Людмила Гаврина,8431007,Людмила Гаврина,jzc3xb,other2,NaN,NaN,NaN,NaN,NaN,2021-05-18
1968,17nukae,1. недозвон 1 сутки,2713,2021-07-21 12:36:38.707,custom,NaN,NaN,NaN,Татьяна Чернышева,17nukae,fb_leads,fb_leads,NaN,NaN,NaN,NaN,2021-07-21
3931,h36qpu,9. есть оплата и запись,150792.0,2021-04-30 17:15:36.473,custom,h36qpu,Ася Абухова,7446028,Ася Абухова,h36qpu,facebook,NaN,NaN,NaN,NaN,NaN,2021-04-13
2600,p3d066,1. недозвон 1 сутки,449,2021-07-15 09:52:18.809,custom,NaN,NaN,NaN,Ян Фаткуллин,p3d066,leto landing,NaN,NaN,NaN,NaN,0,2021-07-14
3928,h11gfp,отказ или отвал,95267.0,2021-04-11 15:20:20.434,closed,NaN,NaN,NaN,Яков,h11gfp,facebook,NaN,NaN,NaN,NaN,7,2021-04-11
4989,18c0vpb,5.встреча подтверждена,65.0,2021-07-31 15:38:49.994,custom,NaN,NaN,NaN,Маргарита,18c0vpb,fb_leads,fb_leads,NaN,NaN,NaN,NaN,2021-07-26
5755,zkmmg7,1. недозвон (пингуем нед.),11521.0,2021-07-06 11:50:52.048,custom,NaN,NaN,NaN,| ПЕРМАНЕНТ ТАТУАЖ | СПБ |,zkmmg7,fb_leads,fb_leads,NaN,NaN,NaN,0,2021-07-03
6118,83zx3z,недозвон (нет контакта),92.0,2021-06-21 11:41:56.780,custom,NaN,NaN,NaN,Оксана Литвак,83zx3z,other,NaN,NaN,NaN,NaN,0,2021-04-29


In [30]:
#добавим мэппинг статусов из файла
statuses = pd.read_excel('mappings.xlsx', engine='openpyxl', sheet_name='statuses', index_col=1)
fin_df = fin_df.merge(statuses, left_on='status', right_index=True, how='left')
fin_df.head(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,...,utm_campaign,utm_term,utm_medium,utm_content,причина отказа,Дата заявки,этап,Созвон,Бронирование,Встреча
0,119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open,119e7p5,Светлана (Иван),10994925,Светлана (Иван),119e7p5,...,NaN,NaN,NaN,NaN,NaN,2021-07-06,0 заявка,0.0,0.0,0.0
1,119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open,NaN,NaN,NaN,Александра Борисова,119eteq,...,fb_leads,NaN,NaN,NaN,11,2021-07-07,0 заявка,0.0,0.0,0.0
2,119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open,NaN,NaN,NaN,КАТЕРИНА,119fjhx,...,fb_leads,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
3,119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open,119g3bp,Анна(Денис),11200929,Анна(Денис),119g3bp,...,NaN,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
4,119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open,NaN,NaN,NaN,Марина Белькова,119gcy3,...,fb_leads,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
5,119gmeg,0. новый лид,303,2021-07-07 05:00:28.312,open,NaN,NaN,NaN,Светлана Керебко,119gmeg,...,fb_leads,NaN,NaN,NaN,15,2021-07-07,0 заявка,0.0,0.0,0.0
6,119gr05,0. новый лид,277,2021-07-07 05:28:11.314,open,NaN,NaN,NaN,Светлана Реутова,119gr05,...,fb_leads,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
7,119h0p7,0. новый лид,174,2021-07-07 07:20:50.794,open,119h0p7,Галина Пеншина,11102355,Галина Пеншина,119h0p7,...,fb_leads,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
8,119h459,0. новый лид,139,2021-07-07 07:52:26.718,open,NaN,NaN,NaN,светлана иванова 888,119h459,...,fb_leads,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
9,119h57w,0. новый лид,128,2021-07-20 19:34:56.653,open,NaN,NaN,NaN,Светлана Андреевская,119h57w,...,fb_leads,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0


In [31]:
#создадим отдельные столбцы с датой созвона, брони и встречи
fin_df['call'] = fin_df['start_dt'].where(fin_df['Созвон']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['meeting'] = fin_df['start_dt'].where(fin_df['Встреча']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['booking'] = fin_df['start_dt'].where(fin_df['Бронирование']==1, np.NaN).groupby(fin_df['index']).transform('min')
#выкинем лишние столбцы
fin_df = fin_df.drop(columns=['Созвон', 'Встреча', 'par_name', 'index', 'par_id']).rename(columns={'причина отказа': 'churn_reason'})
fin_df['lead_date'] = fin_df.groupby('standard_id')['start_dt'].transform('min')
fin_df['lead_date'] = fin_df['Дата заявки'].combine_first(fin_df['lead_date'])
#конвертнем дату заявки в datetime, предварительно проверив на пустоту
if len(fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date']) > 0:
    fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date'] = fin_df[fin_df['Дата заявки'].isna()]['lead_date'].apply(pd.to_datetime).dt.date
fin_df = fin_df.drop(columns=['Дата заявки', 'Бронирование'])
fin_df = fin_df.rename(columns={'этап': 'etap'})
fin_df.sample(7)

,status,min_passed,start_dt,type,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_term,utm_medium,utm_content,churn_reason,etap,call,meeting,booking,lead_date
4778,1. недозвон (пингуем нед.),9837.0,2021-07-12 18:06:58.797,custom,NaN,Мария,139j8da,fb_leads,fb_leads,NaN,NaN,NaN,0,1 устанавливаем контакт,NaT,NaT,NaT,2021-07-10
3357,4. после брони 1 недозвон,100.0,2021-07-13 12:01:09.073,custom,11157305,Елена (Иосиф),11bjdrg,fb_leads,fb_leads,NaN,NaN,NaN,NaN,4 подтверждаем встречу,2021-07-12 10:38:42.333,2021-07-13 13:41:24.500,2021-07-12 10:38:42.333,2021-07-08
1185,0. новый лид,727,2021-08-08 21:02:18.911,open,NaN,Natalia Saratova,rp12pa,fb,math_fb_leadgen_september_offer,NaN,cpc,offer3_zapishis_v_september,NaN,0 заявка,2021-08-09 12:24:04.814,NaT,NaT,2021-08-08
7130,7. отказ после встречи,17124.0,2021-08-01 13:03:40.153,custom,NaN,Катерина,18bvjx2,,NaN,NaN,NaN,NaN,NaN,5 проводим встречу,2021-07-26 15:15:28.485,2021-07-29 12:44:21.540,2021-07-26 15:15:28.485,2021-07-25
4430,1. недозвон 2 сутки,5548.0,2021-06-13 16:19:25.596,custom,NaN,Екатерина Баландина (Даня),x18kyw,fb_leads,fb_leads,NaN,NaN,NaN,0,1 устанавливаем контакт,2021-06-20 17:10:24.005,2021-06-22 23:04:22.483,2021-06-20 17:10:24.005,2021-06-03
1478,0. новый лид,622,2021-06-28 22:57:32.803,open,NaN,Людмила (FB leads via Zapier),z7c57v,fb_leads,fb_leads,NaN,NaN,NaN,0,0 заявка,2021-07-03 15:27:32.661,2021-07-06 15:51:14.666,2021-07-05 12:13:58.033,2021-06-28
3726,1. недозвон 2 сутки,488.0,2021-07-31 10:04:15.977,custom,NaN,Мария Столбова,1abzrta,fb_leads,fb_leads,NaN,NaN,NaN,NaN,1 устанавливаем контакт,NaT,NaT,NaT,2021-07-29


In [32]:
fin_df['standard_id'].nunique()

1658

In [33]:
fin_df.to_csv('fin1_df.csv')

# ЗЭ ЭНД